In [1]:
import wikipedia # libreria de python de wikipedia
from datetime import date # para saber el anio actual
import re #se usa para poder reemplazar los simbolos por espacios en un string
import requests
import json


In [2]:
# CONSTANTES

wikipedia.set_lang("es") # setea el lenguaje en que se va a buscar en wikipedia

todays_date = date.today()

meses_nombres = {'enero': 1,'febrero': 2,'marzo': 3,'abril': 4,'mayo': 5,'junio': 6,'julio': 7,'agosto': 8,'septiembre': 9,'octubre': 10,'noviembre': 11,'diciembre': 12}
key_list = list(meses_nombres.keys())

signo = ("Capricornio", "Acuario", "Piscis", "Aries", "Tauro", "Géminis", "Cáncer", "Leo", "Virgo", "Libra", "Escorpio", "Sagitario")
fechas = (20, 19, 20, 20, 21, 21, 22, 22, 22, 22, 22, 21)

WIKI_REQUEST_IMAGEN = 'http://en.wikipedia.org/w/api.php?action=query&prop=pageimages&format=json&piprop=original&titles='
WIKI_REQUEST_INFOBOX = 'https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles='



In [3]:
# funcion que devuelve el numero del mes que se ingresa y el nombre

def devolver_nombre_mes(num):
    return key_list[num-1] # busca el mes dentro del diccionario y se guarda el nombre


In [4]:
# devuelve el signo del zodiaco segun dia y mes

def calcular_signo(mes_numero,dia):
    mes_numero=mes_numero-1
    if dia>fechas[mes_numero]:
        mes_numero=mes_numero+1
    if mes_numero==12:
        mes_numero=0
    signo_persona = signo[mes_numero] # guarda en una variable el signo de la persona
    return signo_persona

In [5]:
#calcula la edad de la persona

def calcular_edad(anio):
    edad = todays_date.year - anio #resta anio actual menos anio de nacimiento
    return edad

In [6]:
# devuelve todas las coincidencias

persona = input('Ingrese persona: ')

lista_de_coincidencias = wikipedia.search(persona)  #busca todas las coincidencias que exiten en wikipedia con el nombre

for i in range(0,len(lista_de_coincidencias)): # imprime todas las coincidencias de wikipedia con el nombre ingresado
    print('Opcion',i,":",lista_de_coincidencias[i])

Ingrese persona: messi
Opcion 0 : Lionel Messi
Opcion 1 : Rivalidad Cristiano-Messi
Opcion 2 : Messi (película de 2014)
Opcion 3 : José de Yanguas Messía
Opcion 4 : Messi (desambiguación)
Opcion 5 : Messia-sur-Sorne
Opcion 6 : José Luis Messía
Opcion 7 : Marcelo Messias
Opcion 8 : FIFA/FIFPro World XI
Opcion 9 : Fabrício dos Santos Messias


In [7]:
# se queda con la pagina de la persona ingresada

opcion = int(input('Ingrese Opcion: '))
pagina = wikipedia.WikipediaPage(lista_de_coincidencias[opcion]) # se queda con la pagina de wikipedia de la persona

Ingrese Opcion: 0


In [8]:
## EMPIEZA A CALCULAR LAS FECHAS

response  = requests.get(WIKI_REQUEST_INFOBOX + pagina.title)
json_data = json.loads(response.text)
json_infobox = list(json_data['query']['pages'].values())[0]['revisions'][0]['*']
info_box = json_infobox.split("Infobox")[1] # se queda con la info de la seccion "infobox" de wikipedia

In [9]:
## ---------- NACIMIENTO ----------- ##
birth_date = json_infobox.split("birth_date")[1]
birth_date = birth_date.split("}")[0] # se queda con la fecha de cumpleaños de la persona

birth_date = re.sub('[^a-zA-Z0-9\n\.]', ' ', birth_date) # cambia todos los simbolos por espacios

numeros_fechas_cumple = [int(i) for i in birth_date.split() if i.isdigit()] # se queda con todos los numeros del string

anio_nacimiento = numeros_fechas_cumple[0]
mes_nacimiento = numeros_fechas_cumple[1]
dia_nacimiento = numeros_fechas_cumple[2]

print(dia_nacimiento,mes_nacimiento,anio_nacimiento)

24 6 1987


In [10]:
## ----------- MUERTE -------------- ##
try: 
    birth_date = json_infobox.split("death_date")[1]
    birth_date = birth_date.split("}")[0]
    birth_date=re.sub('[^a-zA-Z0-9\n\.]', ' ', birth_date) # cambia todos los simbolos por espacios
    numeros_fechas_muerte = [int(i) for i in birth_date.split() if i.isdigit()] # se queda con todos los numeros del string

    anio_muerte = numeros_fechas_muerte[0]
    mes_muerte = numeros_fechas_muerte[1]
    dia_muerte = numeros_fechas_muerte[2]

    print(dia_muerte,mes_muerte,anio_muerte)
    
except:
    print('La persona no murió.')
    anio_muerte = 0
    

La persona no murió.


In [11]:
## ---------- IMAGEN --------------- ##

response  = requests.get(WIKI_REQUEST_IMAGEN + pagina.title)
json_data = json.loads(response.text)
imagen_persona = list(json_data['query']['pages'].values())[0]['original']['source']

print(imagen_persona)

https://upload.wikimedia.org/wikipedia/commons/c/c1/Lionel_Messi_20180626.jpg


In [12]:
# dice toda la informacion de la persona

mes_nacimiento_nombre = devolver_nombre_mes(mes_nacimiento)
edad = calcular_edad(anio_nacimiento)
signo = calcular_signo(mes_nacimiento,dia_nacimiento)

print('---------- FICHA DE LA PERSONA ----------')

print('Nombre:',pagina.title) #titulo de la pagina de wikipedia

print('Fecha de Nacimiento:',dia_nacimiento,"de",mes_nacimiento_nombre,'de',anio_nacimiento)

if (anio_muerte != 0):
      mes_muerte_nombre = devolver_nombre_mes(mes_muerte)
      print('Fecha de Muerte:',dia_muerte,"de",mes_muerte_nombre,'de',anio_muerte)
      print('La persona tendría:',edad,'años.')
else:
    print('Edad:',edad)
    
print('Signo del Zodíaco:',signo)

print('Información extraída de:',pagina.url)

print('Imagen de la persona:',imagen_persona)

---------- FICHA DE LA PERSONA ----------
Nombre: Lionel Messi
Fecha de Nacimiento: 24 de junio de 1987
Edad: 34
Signo del Zodíaco: Cáncer
Información extraída de: https://es.wikipedia.org/wiki/Lionel_Messi
Imagen de la persona: https://upload.wikimedia.org/wikipedia/commons/c/c1/Lionel_Messi_20180626.jpg
